### To save it in drive

In [97]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [98]:
my_local_drive= '/content/drive/My Drive/TER_M1/'
import sys
sys.path.append(my_local_drive)
%cd $my_local_drive

/content/drive/My Drive/TER_M1


# The main project

In [ ]:
####  Load imports
import os
import string
import matplotlib
import matplotlib_inline
import numpy as np
import pandas as pd
import gensim.models
import nltk
import matplotlib.pyplot as plt
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

matplotlib_inline
nltk.download('stopwords')
nltk.download('wordnet')

 In order to process this data model, we propose to consider each table as a symbol document and the set of tables as a collection of documents. A symbol can be a table’s name, a table’s attribute, or a relationship between two table (foreign key).

We model it by repeating (4 times in this case) the foreign key and the name of the source document in the target documents. We do the same for the name of the target documents in the source document.


In [113]:
# ........... read documents.........................
data = open("document_construction.txt", "r")

### Preprocessing

In [114]:
#    stop words
stop_words = set(stopwords.words('english'))

# punctuation
punctuation = set(string.punctuation)

# lemmatization

lemmatization = WordNetLemmatizer()

In [115]:
# function clean
def clean(documents):
    # split documents and remove stop words

    split_doc = " ".join([i for i in documents.lower().split() if i not in stop_words])

    # remove punctuation
    punc_doc = ''.join([j for j in split_doc if j not in punctuation])

    # normalize the text
    normalized = " ".join([lemmatization.lemmatize(word) for word in punc_doc.split()])

    return normalized

In [116]:
# clean documents in the file text

corpus = data
clean_documents = [clean(doc) for doc in corpus]

In [117]:
# Tables names 
Table_name = []

for doc in clean_documents:
     d=0
     if d==0:

       t= doc.split()[0] +" " +doc.split()[1]
       if doc.split()[0]!= doc.split()[1]:

           Table_name.append(t)
       else:
           Table_name.append(doc.split()[0])
       d=d+1

### CLUSTERING

TF-IDF Vectorization

In [120]:
tfidf_vectorizer = TfidfVectorizer(lowercase=False)

tf_idf = tfidf_vectorizer.fit_transform(clean_documents)
tf_idf

<14x96 sparse matrix of type '<class 'numpy.float64'>'
	with 162 stored elements in Compressed Sparse Row format>

In [ ]:
#  view ressult
feature_names = tfidf_vectorizer.get_feature_names()
dense = tf_idf.todense()
denselist = dense.tolist()

all_key_words = []

for d in denselist:
    i = 0
    kwords = []

    for word in d:
        if word > 0:
            kwords.append(feature_names[i])
        i = i + 1
    all_key_words.append(kwords)

Elbow Method

In [ ]:
wcss = []  # WCSS Scores = within-cluster-sum of squared distance
clusters_1 = range(1, 12)
for i in clusters_1:
    kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=30, algorithm="elkan")
    kmeans.fit(tf_idf)
    wcss.append(kmeans.inertia_)

In [ ]:
plt.plot(clusters_1, wcss)
plt.title('Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('Sum of Squared Distance')
plt.show()

Implementation of KMeans

In [ ]:
clusters = 5  # modified for Petclinc data will be 3 or 2

model = KMeans(n_clusters=clusters, init="k-means++", max_iter=300, n_init=30, algorithm="elkan")

model.fit(tf_idf)

order_centroids = model.cluster_centers_.argsort()[:, ::-1]

terms = tfidf_vectorizer.get_feature_names()

labels = model.labels_

In [ ]:
for i in range(clusters):
  print(f" \n Cluster {i} \n")
  for ind in order_centroids[i, :10]:
      print(terms[ind])

In [ ]:
#  Obtained Clusters

obtainesCluster =[]
print("Obtained Clusters: \n")
for c in range(clusters):
    print(" Cluster", c)
    print("__________________________________________")
    count = 0
    cluseterI =[]
    cluseterI.append(c)
    for i in labels:

        if i == c:
            print('Table: {} ---> Document {}'.format(Table_name[count], count))
            cluseterI.append(Table_name[count])
        count = count + 1

    obtainesCluster.append(cluseterI)
    print("__________________________________________")

In [ ]:
print(" test:")
cc=0
for oc in obtainesCluster:
    print("\n cluseter",cc," -->",oc)
    cc= cc +1

Using PCA to reduces the dimensionality of a data set to an arbitrary number while preserving most of the information contained in it

In [ ]:
pca = PCA(n_components=2)

scatter_plot_points = pca.fit_transform(tf_idf.toarray())

colors = ["r", "b", "c", "y", "m", "g"]

X_axe = [i[0] for i in scatter_plot_points]
Y_axe = [i[1] for i in scatter_plot_points]

plt.scatter(X_axe, Y_axe, c=[colors[d] for d in labels])
plt.show()

Using seaborn visualization


In [ ]:
import seaborn as sns

plt.figure(figsize=(10, 6))     # set image size
plt.title("Clusters", fontdict={"fontsize": 18})

# set axes names
plt.xlabel("X0", fontdict={"fontsize": 16})
plt.ylabel("X1", fontdict={"fontsize": 16})

sns.scatterplot(data=tf_idf.toarray(), x=X_axe , y=Y_axe, palette="viridis",c=[colors[d] for d in labels])
plt.show()